In [ ]:
#from google.colab import drive
#drive.mount("/content/drive/")

Mounted at /content/drive/


In [95]:
!pip install tensorflow==2.9.1
!pip install keras==2.9.0

In [32]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense,Convolution2D,MaxPooling2D,Flatten
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.applications import VGG16

In [33]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_e9a3d7cf91cf4bd0baf7ab4703a8d4df = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='7cp13pz8yd9to4kxml25Jjyb1MGFNXZn8TS5aplui_J3',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

streaming_body_1 = client_e9a3d7cf91cf4bd0baf7ab4703a8d4df.get_object(Bucket='insuranceproject-donotdelete-pr-ugqohp6gzqcw3h', Key='Dataset.zip')['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [34]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [35]:
pwd

'/home/wsuser/work'

In [36]:
import os
os.listdir("/home/wsuser/work/Dataset")

['training', 'validation']

In [37]:
image_generator=ImageDataGenerator(vertical_flip=False,horizontal_flip=True,shear_range=0.1,zoom_range=0.1,rescale=1/255,brightness_range=(0.2,0.8))

In [38]:
X_train1=image_generator.flow_from_directory(target_size=(224,224),
                                    directory="/home/wsuser/work/Dataset/training",
                                    class_mode="categorical",
                                   batch_size=10,
                                            subset="training")

Found 979 images belonging to 3 classes.


In [39]:
image_generator_1=ImageDataGenerator(vertical_flip=False,horizontal_flip=True,shear_range=0.1,zoom_range=0.1,rescale=1/255,brightness_range=(0.2,0.8))

In [40]:
X_test1=image_generator_1.flow_from_directory(target_size=(224,224),
                                    directory="/home/wsuser/work/Dataset/validation",
                                    class_mode="categorical",
                                   batch_size=10,
                                )

Found 171 images belonging to 3 classes.


In [41]:
vgg16=VGG16(include_top=False,input_shape=(224,224,3),weights='imagenet')
for i in vgg16.layers:
  i.trainable=False

In [42]:
flatten_layer=Flatten()(vgg16.output)
output=Dense(3,activation="softmax")(flatten_layer)
model1=Model(inputs=vgg16.input,outputs=output)


In [43]:
model1.compile(loss=CategoricalCrossentropy(),
              optimizer="adam",
              metrics=["accuracy"])

In [44]:
model1.fit(X_train1,validation_data=X_test1,epochs=25,steps_per_epoch=97,validation_steps=17)

Epoch 1/25
97/97 [==============================] - 388s 4s/step - loss: 1.3577 - accuracy: 0.4850 - val_loss: 1.1156 - val_accuracy: 0.5118
Epoch 2/25
97/97 [==============================] - 383s 4s/step - loss: 0.7778 - accuracy: 0.6708 - val_loss: 1.2616 - val_accuracy: 0.5647
Epoch 3/25
97/97 [==============================] - 390s 4s/step - loss: 0.7692 - accuracy: 0.6780 - val_loss: 1.0460 - val_accuracy: 0.5647
Epoch 4/25
97/97 [==============================] - 387s 4s/step - loss: 0.6360 - accuracy: 0.7245 - val_loss: 1.0037 - val_accuracy: 0.5647
Epoch 5/25
97/97 [==============================] - 392s 4s/step - loss: 0.5284 - accuracy: 0.7833 - val_loss: 0.8591 - val_accuracy: 0.6706
Epoch 6/25
97/97 [==============================] - 383s 4s/step - loss: 0.4751 - accuracy: 0.8132 - val_loss: 0.9704 - val_accuracy: 0.6529
Epoch 7/25
97/97 [==============================] - 384s 4s/step - loss: 0.4887 - accuracy: 0.8008 - val_loss: 1.3857 - val_accuracy: 0.5647
Epoch 8/25
97

In [45]:
model1.save("Model1.h5")

In [46]:
!tar -zcvf location_model.tgz Model1.h5

Model1.h5


In [47]:
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 16.5 MB/s eta 0:00:01


In [74]:
cred={"url":"https://us-south.ml.cloud.ibm.com",
 "apikey":"F-FjystdF3aQ-G67Twcfyk0SRAukYiIFiB5g9QzJNkKT"}

In [75]:
from ibm_watson_machine_learning import APIClient

In [76]:
client=APIClient(cred)

In [86]:
def guid_from_space_name(client, space_name): 
    space=client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']['name'] == space_name)['metadata']['id'])

In [87]:
space_uid=guid_from_space_name (client, 'Models') 
print("Space UID= " + space_uid)

Space UID= 11cba668-4b75-436d-af79-82c7d3905230


In [88]:
client.set.default_space(space_uid)

'SUCCESS'

In [96]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
runtime-22.1-py3.9             12b83a1

In [107]:
software_spec_uid=client.software_specifications.get_uid_by_name("tensorflow_1.15-py3.6")

In [108]:
model_details=client.repository.store_model(model='location_model.tgz',meta_props={client.repository.ModelMetaNames.NAME:"CNN", 
                                                                                   client.repository.ModelMetaNames.TYPE: "keras_2.2.4", 
                                                                                   client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: software_spec_uid})
model_id=client.repository.get_model_uid(model_details)

Failure during creating model. (POST https://us-south.ml.cloud.ibm.com/ml/v4/models?version=2021-06-24&space_id=11cba668-4b75-436d-af79-82c7d3905230)
Status code: 400, body: {
  "trace": "0f3f80cc0797e7e1c534a9e90e267191",
  "errors": [{
    "code": "invalid_request_entity",
    "message": "Invalid request entity: Unsupported software specification 'tensorflow_1.15-py3.6'.",
    "more_info": "https://cloud.ibm.com/apidocs/machine-learning"
  }],
  "status_code": "400"
}


ApiRequestFailure: Failure during creating model. (POST https://us-south.ml.cloud.ibm.com/ml/v4/models?version=2021-06-24&space_id=11cba668-4b75-436d-af79-82c7d3905230)
Status code: 400, body: {
  "trace": "0f3f80cc0797e7e1c534a9e90e267191",
  "errors": [{
    "code": "invalid_request_entity",
    "message": "Invalid request entity: Unsupported software specification 'tensorflow_1.15-py3.6'.",
    "more_info": "https://cloud.ibm.com/apidocs/machine-learning"
  }],
  "status_code": "400"
}